In [ ]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

In [ ]:
#SET WORKING DIRECTORY for each data ************
os.chdir('paste directory name here')

In [ ]:
cv2.saliency

In [ ]:
#CHANGE FILENAMES according to image type analyzed
imagetype = "Building1.png"
fixdf_name = "result10.20_building1.xlsx"
imtype = "Building1"

In [ ]:
#import all required files

#Import fixation data
fixdf = pd.read_excel(fixdf_name, sheet_name="fixation")

In [ ]:
#change name of the file import accordingly to read the picture without eye gaze trace
if __name__ == "__main__" :
    image = cv2.imread(os.path.join("paste directory here", imagetype))
    #use opencv static spectral residual saliency mapping
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliencyMap) = saliency.computeSaliency(image)
    saliencyMap = (saliencyMap * 255).astype("uint8")
    
#create a df of the saliency map
    if success is True:
        df = pd.DataFrame(saliencyMap)

In [ ]:
print(fixdf)

In [ ]:
#extract x/z and y/z fixation coordinates from the fixdf dataframe
fixations = fixdf[["x","y"]]

In [ ]:
print(fixations)

In [ ]:
#define an equation to convert x/z and y/z to pixels compatible to threshMap (px_x and px_y)
def xytoPixel(row):
    if row['x'] < 0 and row['y'] >= 0:
        px_x = 2048/2 - (900 * abs(row['x'])) 
        px_y = 1024/2 - (900 * abs(row['y']))
    elif row['x'] < 0 and row['y'] < 0:
        px_x = 2048/2 - (900 * abs(row['x'])) 
        px_y = 1024/2 + (900 * abs(row['y']))
    elif row['x'] >= 0 and row['y'] >= 0:
        px_x = 2048/2 + (900 * abs(row['x'])) 
        px_y = 1024/2 - (900 * abs(row['y']))
    elif row['x'] >= 0 and row['y'] < 0:
        px_x = 2048/2 + (900 * abs(row['x'])) 
        px_y = 1024/2 + (900 * abs(row['y']))
    else :
        px_x = 'error'
        px_y = 'error'
    return pd.Series({'px_x': px_x, 'px_y': px_y})

In [ ]:
#Create an independent copy of fixations df to separate it as a slice of bigger df
fixations = fixations.copy()

# Apply the xytoPixel function and assign results to new columns
fixations[['px_x', 'px_y']] = fixations.apply(xytoPixel, axis=1)

# Display the updated DataFrame
print(fixations)

In [ ]:
#Round up the pix_x and pix_y to obtain the closest pixel coordinates
def roundUp(row):
    pxru_x = np.round(row['px_x'])
    pxru_y = np.round(row['px_y'])
    return pd.Series({'pxru_x': pxru_x, 'pxru_y': pxru_y})

fixations[['pxru_x', 'pxru_y']] = fixations.apply(roundUp, axis=1)
fixations[['pxru_x']] = fixations[['pxru_x']].astype(int)
fixations[['pxru_y']] = fixations[['pxru_y']].astype(int)

#Work out the binary value for each pxru_x and pxru_y coordinates according to df (finegrained saliency map)
fixations['saliency_value'] = df.to_numpy()[fixations['pxru_y'], fixations['pxru_x']]

print(fixations)

In [ ]:
plot_width = 2048
plot_height = 1024

plt.figure(figsize=(10, 5))

# Display the traceimage in the background. Use extent to align the image with plot axis of size 2048 x 1024
plt.imshow(saliencyMap, extent=[0, plot_width, plot_height, 0])

plt.scatter(fixations['px_x'], fixations['px_y'], color='red', label='Fixation Points')
plt.xlim(0, plot_width)
plt.ylim(plot_height, 0)
plt.xlabel('X Coordinate (pixels)')
plt.ylabel('Y Coordinate (pixels)')
plt.grid(False)
plt.legend()

#save overlapped images as png. Check with FOVE0 output scanpath photo and double check to see if the fixations match scanpath
plt.savefig(imtype + 'saliencyplot.png', dpi=800)
plt.show()

In [ ]:
#save to excel file
fixations.to_excel(imtype + '_saliencyvalue.xlsx') 